# Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import random
import gc

from pandas import *

# SK-learn libraries.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import re

# scipy
from scipy.signal import butter, lfilter


from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.datasets import make_classification




# Import and Merge The Data

In [2]:
#Remove the channels we don't want 
def Remove_Channels(df):
    df.drop(df.columns[[15,16,20,21,22,25,26,27,28,29,30,31,32]], axis=1, inplace=True)
    return df



path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
train_data_filenames = glob.glob(path + "/subj1_series[1-2]*data.csv") #load only subject 1, series 1-3
list_ = []
Train_Array_Lengths = []
Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_data_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(Remove_Channels(df))
    Train_Array_Lengths.append(len(df))
train_data = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#now with event data
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
train_event_filenames = glob.glob(path + "/subj1_series[1-2]*events.csv") #load only subject 1, series 1-3
list_ = []

Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_event_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
train_events = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#do the same thing and now get your testing data
test_data_filenames = glob.glob(path + "/subj1_series3*data.csv") #load only subject 1, series 1-3
list_ = []
Test_Array_Lengths = []
Start_Time = time.time()
print time.time(), "Merging..."

for file_ in test_data_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(Remove_Channels(df))
    Test_Array_Lengths.append(len(df))
test_data = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#now with event data
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
test_event_filenames = glob.glob(path + "/subj1_series3*events.csv") #load only subject 1, series 1-3
list_ = []

Start_Time = time.time()
print time.time(), "Merging..."

for file_ in test_event_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
test_events = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"

1438813062.47 Merging...
1438813062.47 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series1_data.csv
1438813063.17 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series2_data.csv
Merge Complete: 2.55599999428 total seconds
1438813065.04 Merging...
1438813065.04 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series1_events.csv
1438813065.23 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series2_events.csv
Merge Complete: 0.526000022888 total seconds
1438813065.57 Merging...
1438813065.57 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series3_data.csv
Merge Complete: 1.09799981117 total seconds
1438813066.67 Merging...
1438813066.67 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series3_events.csv
Merge Complete: 0.238999843597 total seconds


# Separate the data into the 6 events and into training and test sets

In [3]:
Train_Labels_HandStart =  train_events['HandStart'].to_sparse(fill_value=0)
Train_Labels_FirstDigitTouch =  train_events['FirstDigitTouch'].to_sparse(fill_value=0)
Train_Labels_BothStartLoadPhase =  train_events['BothStartLoadPhase'].to_sparse(fill_value=0)
Train_Labels_LiftOff =  train_events['LiftOff'].to_sparse(fill_value=0)
Train_Labels_Replace =  train_events['Replace'].to_sparse(fill_value=0)
Train_Labels_BothReleased =  train_events['BothReleased'].to_sparse(fill_value=0)
# Train_Labels_Combined = train_events.HandStart.map(str) + ',' + train_events.FirstDigitTouch.map(str) + ',' + train_events.BothStartLoadPhase.map(str) + ',' + train_events.LiftOff.map(str) + ',' + train_events.Replace.map(str) + ',' + train_events.BothReleased.map(str)


Train_Data = train_data.iloc[0:,1:] # select only column data


Test_Labels_HandStart =  test_events['HandStart'].to_sparse(fill_value=0)
Test_Labels_FirstDigitTouch =  test_events['FirstDigitTouch'].to_sparse(fill_value=0)
Test_Labels_BothStartLoadPhase =  test_events['BothStartLoadPhase'].to_sparse(fill_value=0)
Test_Labels_LiftOff =  test_events['LiftOff'].to_sparse(fill_value=0)
Test_Labels_Replace =  test_events['Replace'].to_sparse(fill_value=0)
Test_Labels_BothReleased =  test_events['BothReleased'].to_sparse(fill_value=0)


Test_Data = test_data.iloc[0:,1:] # select only data columns

#delete what isn't needed anymore to save memory. 
del train_data
del test_data
del train_events
del test_events
gc.collect()

101

# Define functions for feature extraction

In [15]:

#bin the time 
def Bin_Time(num_rows,num_bins):
    Bin_Size = num_rows/num_bins
    Bins = np.zeros(shape=(num_rows,num_bins))
    Bin_Min = 0
    Bin_Max = Bin_Size
    for i in range(0,num_bins):
        Bins[Bin_Min:Bin_Max,i] = 1
        Bin_Min = Bin_Min + Bin_Size
        Bin_Max = Bin_Max + Bin_Size
    return Bins


##filters borrowed from Nihar. 
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


#run PCA and return the number of PCs that explain the given amount of variance. 
def extractFeatures_PCA(Train_Data,Test_Data, PercentVarExplained):
    pca = PCA()
    pca.fit(Train_Data)  
    
    Explained_Variance_Ratios = pca.explained_variance_ratio_
    for i in range(1,len(Explained_Variance_Ratios)):
        if sum(Explained_Variance_Ratios[0:i]) >= PercentVarExplained:
                   NumPCs = i + 1 #add 1 since numpy array ranges are not inclusive
                   break
    PCA_Projections = pca.transform(Train_Data)[:,0:NumPCs]
    PCA_Projections_Test = pca.transform(Test_Data)[:,0:NumPCs]
    return np.float32(PCA_Projections),np.float32(PCA_Projections_Test)



#return rolling mean of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_mean(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_mean(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'mean' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling variance of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_var(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_var(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'var' + str(i)})
    return np.array(new_df.astype('float32'))


#return rolling min of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_min(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_min(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'min' + str(i)})
    return np.array(new_df.astype('float32'))


#return rolling max of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_max(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_max(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'max' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling skew of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_skew(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_skew(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'skew' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling skew of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_kurt(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_kurt(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'kurt' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling quantile of each column in a pandas dataframe with a given window and quantile. Returns df of same size. 
def df_rolling_quantile(df,window,quantile):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_quantile(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'Pct'+ quantile + "_" + str(i)})
    return np.array(new_df.astype('float32'))


#BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
#return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_corr(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(pd.rolling_corr(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)



#BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
#return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_cov(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(rolling_cov(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)




#Feature Creation 

This section takes rolling statistics for different windows of time for each column of the dataframe. It then does an RFE feature selection technique to keep only the top features for each rolling stat for each of the six labels. It then appends all of this together into a customized feature array for each of the six labels.  

This took about 10 minutes on my machine

In [5]:

HandStart_Test_Features = []
FirstDigitTouch_Test_Features = []
BothStartLoadPhase_Test_Features = []
LiftOff_Test_Features = []
Replace_Test_Features = []
BothReleased_Test_Features = []

HandStart_Train_Features = []
FirstDigitTouch_Train_Features = []
BothStartLoadPhase_Train_Features = []
LiftOff_Train_Features = []
Replace_Train_Features = []
BothReleased_Train_Features = []

Windows = [20, 100]
Start_Time = time.time()
for w in Windows:
    Raw_Data_Rolling_Window = w
    
    
    RawData_Rolling_Means_Train = df_rolling_mean(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Var_Train = df_rolling_var(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Skewness_Train = df_rolling_skew(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Min_Train = df_rolling_min(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Max_Train = df_rolling_max(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)


    RawData_Rolling_Means_Test = df_rolling_mean(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Var_Test = df_rolling_var(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Skewness_Test = df_rolling_skew(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Min_Test = df_rolling_min(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Max_Test = df_rolling_max(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)

    print "Calculations With Window of", w, "Complete: ", int(time.time()-Start_Time), " Seconds"

    
    #Append all the features to a feature set for each event. 
    HandStart_Train_Features.append(np.concatenate((RawData_Rolling_Means_Train,RawData_Rolling_Var_Train,RawData_Rolling_Skewness_Train,RawData_Rolling_Min_Train,RawData_Rolling_Max_Train),axis = 1))
    HandStart_Test_Features.append(np.concatenate((RawData_Rolling_Means_Test,RawData_Rolling_Var_Test,RawData_Rolling_Skewness_Test,RawData_Rolling_Min_Test,RawData_Rolling_Max_Test),axis = 1))
    
    FirstDigitTouch_Train_Features.append(np.concatenate((RawData_Rolling_Means_Train,RawData_Rolling_Var_Train,RawData_Rolling_Skewness_Train,RawData_Rolling_Min_Train,RawData_Rolling_Max_Train),axis = 1))
    FirstDigitTouch_Test_Features.append(np.concatenate((RawData_Rolling_Means_Test,RawData_Rolling_Var_Test,RawData_Rolling_Skewness_Test,RawData_Rolling_Min_Test,RawData_Rolling_Max_Test),axis = 1))
    
    BothStartLoadPhase_Train_Features.append(np.concatenate((RawData_Rolling_Means_Train,RawData_Rolling_Var_Train,RawData_Rolling_Skewness_Train,RawData_Rolling_Min_Train,RawData_Rolling_Max_Train),axis = 1))
    BothStartLoadPhase_Test_Features.append(np.concatenate((RawData_Rolling_Means_Test,RawData_Rolling_Var_Test,RawData_Rolling_Skewness_Test,RawData_Rolling_Min_Test,RawData_Rolling_Max_Test),axis = 1))
    
    LiftOff_Train_Features.append(np.concatenate((RawData_Rolling_Means_Train,RawData_Rolling_Var_Train,RawData_Rolling_Skewness_Train,RawData_Rolling_Min_Train,RawData_Rolling_Max_Train),axis = 1))
    LiftOff_Test_Features.append(np.concatenate((RawData_Rolling_Means_Test,RawData_Rolling_Var_Test,RawData_Rolling_Skewness_Test,RawData_Rolling_Min_Test,RawData_Rolling_Max_Test),axis = 1))
    
    Replace_Train_Features.append(np.concatenate((RawData_Rolling_Means_Train,RawData_Rolling_Var_Train,RawData_Rolling_Skewness_Train,RawData_Rolling_Min_Train,RawData_Rolling_Max_Train),axis = 1))
    Replace_Test_Features.append(np.concatenate((RawData_Rolling_Means_Test,RawData_Rolling_Var_Test,RawData_Rolling_Skewness_Test,RawData_Rolling_Min_Test,RawData_Rolling_Max_Test),axis = 1))
    
    BothReleased_Train_Features.append(np.concatenate((RawData_Rolling_Means_Train,RawData_Rolling_Var_Train,RawData_Rolling_Skewness_Train,RawData_Rolling_Min_Train,RawData_Rolling_Max_Train),axis = 1))
    BothReleased_Test_Features.append(np.concatenate((RawData_Rolling_Means_Test,RawData_Rolling_Var_Test,RawData_Rolling_Skewness_Test,RawData_Rolling_Min_Test,RawData_Rolling_Max_Test),axis = 1))
    
    print "Feature Concatenation with a Rolling Window of", w, "Complete: ", int(time.time()-Start_Time), " Seconds"

    

    
#delete what isn't needed anymore to save memory. 

del RawData_Rolling_Means_Train
del RawData_Rolling_Means_Test

del RawData_Rolling_Var_Train
del RawData_Rolling_Var_Test

del RawData_Rolling_Skewness_Train
del RawData_Rolling_Skewness_Test

del RawData_Rolling_Min_Train
del RawData_Rolling_Min_Test

del RawData_Rolling_Max_Train
del RawData_Rolling_Max_Test
gc.collect()



Calculations With Window of 20 Complete:  13  Seconds
Feature Concatenation with a Rolling Window of 20 Complete:  14  Seconds
Calculations With Window of 100 Complete:  28  Seconds
Feature Concatenation with a Rolling Window of 100 Complete:  28  Seconds


214

### Bin the entire series into chunks of time. One feature per time bin. 
Then reduce the time bin feature set for each event with recursive feature elimination. 

In [6]:
Initial_Num_Bins = 100

Reduced_Number_Of_Bin = 10

Reduction_Step = 10

C_Value = 1
penalty = 'l1'
Convergence_tol = .1



Train_Bin_List = []
for i in range (0,len(Train_Array_Lengths)):
    Train_Bin_List.append(Bin_Time(Train_Array_Lengths[i],Initial_Num_Bins)) # bin the time based on the sizes of each series. 
Train_Time_Bins = np.uint8(np.concatenate(Train_Bin_List, axis = 0))


Test_Bin_List = []
for i in range (0,len(Test_Array_Lengths)):
    Test_Bin_List.append(Bin_Time(Test_Array_Lengths[i],Initial_Num_Bins)) # bin the time based on the sizes of each series. 
Test_Time_Bins = np.uint8(np.concatenate(Test_Bin_List, axis = 0))

print "Time Bin Creation Complete: ", int(time.time()-Start_Time), " Seconds"




LogReg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol) 
rfe = RFE(LogReg,Reduced_Number_Of_Bin, step=Reduction_Step)


#reduce Time_Bins to top x features for HandStart
rfe.fit(Train_Time_Bins, Train_Labels_HandStart.to_dense())
HandStart_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
HandStart_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
#reduce Rolling_Means to top x features for FirstDigitTouch
rfe.fit(Train_Time_Bins, Train_Labels_FirstDigitTouch.to_dense())
FirstDigitTouch_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
FirstDigitTouch_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
#reduce Rolling_Means to top x features for BothStartLoadPhase
rfe.fit(Train_Time_Bins, Train_Labels_BothStartLoadPhase.to_dense())
BothStartLoadPhase_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
BothStartLoadPhase_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
#reduce Rolling_Means to top x features for LiftOff
rfe.fit(Train_Time_Bins, Train_Labels_LiftOff.to_dense())
LiftOff_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
LiftOff_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
#reduce Rolling_Means to top x features for Replace
rfe.fit(Train_Time_Bins, Train_Labels_Replace.to_dense())
Replace_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
Replace_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
#reduce Rolling_Means to top x features for BothReleased
rfe.fit(Train_Time_Bins, Train_Labels_BothReleased.to_dense())
BothReleased_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
BothReleased_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set    


del Train_Time_Bins
del Test_Time_Bins

print "Time Bins Added to Feature Sets For Each Event: ", int(time.time()-Start_Time), " Seconds"

Time Bin Creation Complete:  34  Seconds
Time Bins Added to Feature Sets For Each Event:  70  Seconds


In [5]:
#     #reduce Rolling_Means to top x features for HandStart
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_HandStart.to_dense())
#     HandStart_Train_Features.append(rfe.transform(RawData_Rolling_Means_Train)) # Add reduced set of features to full set
#     HandStart_Test_Features.append(rfe.transform(RawData_Rolling_Means_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Means to top x features for FirstDigitTouch
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_FirstDigitTouch.to_dense())
#     FirstDigitTouch_Train_Features.append(rfe.transform(RawData_Rolling_Means_Train)) # Add reduced set of features to full set
#     FirstDigitTouch_Test_Features.append(rfe.transform(RawData_Rolling_Means_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Means to top x features for BothStartLoadPhase
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_BothStartLoadPhase.to_dense())
#     BothStartLoadPhase_Train_Features.append(rfe.transform(RawData_Rolling_Means_Train)) # Add reduced set of features to full set
#     BothStartLoadPhase_Test_Features.append(rfe.transform(RawData_Rolling_Means_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Means to top x features for LiftOff
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_LiftOff.to_dense())
#     LiftOff_Train_Features.append(rfe.transform(RawData_Rolling_Means_Train)) # Add reduced set of features to full set
#     LiftOff_Test_Features.append(rfe.transform(RawData_Rolling_Means_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Means to top x features for Replace
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_Replace.to_dense())
#     Replace_Train_Features.append(rfe.transform(RawData_Rolling_Means_Train)) # Add reduced set of features to full set
#     Replace_Test_Features.append(rfe.transform(RawData_Rolling_Means_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Means to top x features for BothReleased
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_BothReleased.to_dense())
#     BothReleased_Train_Features.append(rfe.transform(RawData_Rolling_Means_Train)) # Add reduced set of features to full set
#     BothReleased_Test_Features.append(rfe.transform(RawData_Rolling_Means_Test)) # Add reduced set of features to full set
    
#     print "Rolling Mean With Window Size ", w, " Complete: ", int(time.time()-Start_Time), " Seconds"
    
    
    
#     #reduce Rolling_Var to top x features for HandStart
#     rfe.fit(RawData_Rolling_Var_Train, Train_Labels_HandStart.to_dense())
#     HandStart_Train_Features.append(rfe.transform(RawData_Rolling_Var_Train)) # Add reduced set of features to full set
#     HandStart_Test_Features.append(rfe.transform(RawData_Rolling_Var_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Var to top x features for FirstDigitTouch
#     rfe.fit(RawData_Rolling_Var_Train, Train_Labels_FirstDigitTouch.to_dense())
#     FirstDigitTouch_Train_Features.append(rfe.transform(RawData_Rolling_Var_Train)) # Add reduced set of features to full set
#     FirstDigitTouch_Test_Features.append(rfe.transform(RawData_Rolling_Var_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Var to top x features for BothStartLoadPhase
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_BothStartLoadPhase.to_dense())
#     BothStartLoadPhase_Train_Features.append(rfe.transform(RawData_Rolling_Var_Train)) # Add reduced set of features to full set
#     BothStartLoadPhase_Test_Features.append(rfe.transform(RawData_Rolling_Var_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Var to top x features for LiftOff
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_LiftOff.to_dense())
#     LiftOff_Train_Features.append(rfe.transform(RawData_Rolling_Var_Train)) # Add reduced set of features to full set
#     LiftOff_Test_Features.append(rfe.transform(RawData_Rolling_Var_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Var to top x features for Replace
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_Replace.to_dense())
#     Replace_Train_Features.append(rfe.transform(RawData_Rolling_Var_Train)) # Add reduced set of features to full set
#     Replace_Test_Features.append(rfe.transform(RawData_Rolling_Var_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Var to top x features for BothReleased
#     rfe.fit(RawData_Rolling_Means_Train, Train_Labels_BothReleased.to_dense())
#     BothReleased_Train_Features.append(rfe.transform(RawData_Rolling_Var_Train)) # Add reduced set of features to full set
#     BothReleased_Test_Features.append(rfe.transform(RawData_Rolling_Var_Test)) # Add reduced set of features to full set
#     print "Rolling Variance With Window Size ", w, " Complete: ", int(time.time()-Start_Time), " Seconds"
    
    

#     #reduce Rolling_Skewness to top x features for HandStart
#     rfe.fit(RawData_Rolling_Skewness_Train, Train_Labels_HandStart.to_dense())
#     HandStart_Train_Features.append(rfe.transform(RawData_Rolling_Skewness_Train)) # Add reduced set of features to full set
#     HandStart_Test_Features.append(rfe.transform(RawData_Rolling_Skewness_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Skewness to top x features for FirstDigitTouch
#     rfe.fit(RawData_Rolling_Skewness_Train, Train_Labels_FirstDigitTouch.to_dense())
#     FirstDigitTouch_Train_Features.append(rfe.transform(RawData_Rolling_Skewness_Train)) # Add reduced set of features to full set
#     FirstDigitTouch_Test_Features.append(rfe.transform(RawData_Rolling_Skewness_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Skewness to top x features for BothStartLoadPhase
#     rfe.fit(RawData_Rolling_Skewness_Train, Train_Labels_BothStartLoadPhase.to_dense())
#     BothStartLoadPhase_Train_Features.append(rfe.transform(RawData_Rolling_Skewness_Train)) # Add reduced set of features to full set
#     BothStartLoadPhase_Test_Features.append(rfe.transform(RawData_Rolling_Skewness_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Skewness to top x features for LiftOff
#     rfe.fit(RawData_Rolling_Skewness_Train, Train_Labels_LiftOff.to_dense())
#     LiftOff_Train_Features.append(rfe.transform(RawData_Rolling_Skewness_Train)) # Add reduced set of features to full set
#     LiftOff_Test_Features.append(rfe.transform(RawData_Rolling_Skewness_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Skewness to top x features for Replace
#     rfe.fit(RawData_Rolling_Skewness_Train, Train_Labels_Replace.to_dense())
#     Replace_Train_Features.append(rfe.transform(RawData_Rolling_Skewness_Train)) # Add reduced set of features to full set
#     Replace_Test_Features.append(rfe.transform(RawData_Rolling_Skewness_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Skewness to top x features for BothReleased
#     rfe.fit(RawData_Rolling_Skewness_Train, Train_Labels_BothReleased.to_dense())
#     BothReleased_Train_Features.append(rfe.transform(RawData_Rolling_Skewness_Train)) # Add reduced set of features to full set
#     BothReleased_Test_Features.append(rfe.transform(RawData_Rolling_Skewness_Test)) # Add reduced set of features to full set
#     print "Rolling Skew With Window Size ", w, " Complete: ", int(time.time()-Start_Time), " Seconds"
    
    
#     #reduce Rolling_Kurtosis to top x features for HandStart
#     rfe.fit(RawData_Rolling_Kurtosis_Train, Train_Labels_HandStart.to_dense())
#     HandStart_Train_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Train)) # Add reduced set of features to full set
#     HandStart_Test_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Kurtosis to top x features for FirstDigitTouch
#     rfe.fit(RawData_Rolling_Kurtosis_Train, Train_Labels_FirstDigitTouch.to_dense())
#     FirstDigitTouch_Train_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Train)) # Add reduced set of features to full set
#     FirstDigitTouch_Test_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Kurtosis to top x features for BothStartLoadPhase
#     rfe.fit(RawData_Rolling_Kurtosis_Train, Train_Labels_BothStartLoadPhase.to_dense())
#     BothStartLoadPhase_Train_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Train)) # Add reduced set of features to full set
#     BothStartLoadPhase_Test_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Kurtosis to top x features for LiftOff
#     rfe.fit(RawData_Rolling_Kurtosis_Train, Train_Labels_LiftOff.to_dense())
#     LiftOff_Train_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Train)) # Add reduced set of features to full set
#     LiftOff_Test_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Kurtosis to top x features for Replace
#     rfe.fit(RawData_Rolling_Kurtosis_Train, Train_Labels_Replace.to_dense())
#     Replace_Train_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Train)) # Add reduced set of features to full set
#     Replace_Test_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Kurtosis to top x features for BothReleased
#     rfe.fit(RawData_Rolling_Kurtosis_Train, Train_Labels_BothReleased.to_dense())
#     BothReleased_Train_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Train)) # Add reduced set of features to full set
#     BothReleased_Test_Features.append(rfe.transform(RawData_Rolling_Kurtosis_Test)) # Add reduced set of features to full set
#     print "Rolling Kurtosis With Window Size ", w, " Complete: ", int(time.time()-Start_Time), " Seconds"
    
       
        
#     #reduce Rolling_Min to top x features for HandStart
#     rfe.fit(RawData_Rolling_Min_Train, Train_Labels_HandStart.to_dense())
#     HandStart_Train_Features.append(rfe.transform(RawData_Rolling_Min_Train)) # Add reduced set of features to full set
#     HandStart_Test_Features.append(rfe.transform(RawData_Rolling_Min_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Min to top x features for FirstDigitTouch
#     rfe.fit(RawData_Rolling_Min_Train, Train_Labels_FirstDigitTouch.to_dense())
#     FirstDigitTouch_Train_Features.append(rfe.transform(RawData_Rolling_Min_Train)) # Add reduced set of features to full set
#     FirstDigitTouch_Test_Features.append(rfe.transform(RawData_Rolling_Min_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Min to top x features for BothStartLoadPhase
#     rfe.fit(RawData_Rolling_Min_Train, Train_Labels_BothStartLoadPhase.to_dense())
#     BothStartLoadPhase_Train_Features.append(rfe.transform(RawData_Rolling_Min_Train)) # Add reduced set of features to full set
#     BothStartLoadPhase_Test_Features.append(rfe.transform(RawData_Rolling_Min_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Min to top x features for LiftOff
#     rfe.fit(RawData_Rolling_Min_Train, Train_Labels_LiftOff.to_dense())
#     LiftOff_Train_Features.append(rfe.transform(RawData_Rolling_Min_Train)) # Add reduced set of features to full set
#     LiftOff_Test_Features.append(rfe.transform(RawData_Rolling_Min_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Min to top x features for Replace
#     rfe.fit(RawData_Rolling_Min_Train, Train_Labels_Replace.to_dense())
#     Replace_Train_Features.append(rfe.transform(RawData_Rolling_Min_Train)) # Add reduced set of features to full set
#     Replace_Test_Features.append(rfe.transform(RawData_Rolling_Min_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Min to top x features for BothReleased
#     rfe.fit(RawData_Rolling_Min_Train, Train_Labels_BothReleased.to_dense())
#     BothReleased_Train_Features.append(rfe.transform(RawData_Rolling_Min_Train)) # Add reduced set of features to full set
#     BothReleased_Test_Features.append(rfe.transform(RawData_Rolling_Min_Test)) # Add reduced set of features to full set 
#     print "Rolling Min With Window Size ", w, " Complete: ", int(time.time()-Start_Time), " Seconds"
    
    
#     #reduce Rolling_Max to top x features for HandStart
#     rfe.fit(RawData_Rolling_Max_Train, Train_Labels_HandStart.to_dense())
#     HandStart_Train_Features.append(rfe.transform(RawData_Rolling_Max_Train)) # Add reduced set of features to full set
#     HandStart_Test_Features.append(rfe.transform(RawData_Rolling_Max_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Max to top x features for FirstDigitTouch
#     rfe.fit(RawData_Rolling_Max_Train, Train_Labels_FirstDigitTouch.to_dense())
#     FirstDigitTouch_Train_Features.append(rfe.transform(RawData_Rolling_Max_Train)) # Add reduced set of features to full set
#     FirstDigitTouch_Test_Features.append(rfe.transform(RawData_Rolling_Max_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Max to top x features for BothStartLoadPhase
#     rfe.fit(RawData_Rolling_Max_Train, Train_Labels_BothStartLoadPhase.to_dense())
#     BothStartLoadPhase_Train_Features.append(rfe.transform(RawData_Rolling_Max_Train)) # Add reduced set of features to full set
#     BothStartLoadPhase_Test_Features.append(rfe.transform(RawData_Rolling_Max_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Max to top x features for LiftOff
#     rfe.fit(RawData_Rolling_Max_Train, Train_Labels_LiftOff.to_dense())
#     LiftOff_Train_Features.append(rfe.transform(RawData_Rolling_Max_Train)) # Add reduced set of features to full set
#     LiftOff_Test_Features.append(rfe.transform(RawData_Rolling_Max_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Max to top x features for Replace
#     rfe.fit(RawData_Rolling_Max_Train, Train_Labels_Replace.to_dense())
#     Replace_Train_Features.append(rfe.transform(RawData_Rolling_Max_Train)) # Add reduced set of features to full set
#     Replace_Test_Features.append(rfe.transform(RawData_Rolling_Max_Test)) # Add reduced set of features to full set
    
#     #reduce Rolling_Max to top x features for BothReleased
#     rfe.fit(RawData_Rolling_Max_Train, Train_Labels_BothReleased.to_dense())
#     BothReleased_Train_Features.append(rfe.transform(RawData_Rolling_Max_Train)) # Add reduced set of features to full set
#     BothReleased_Test_Features.append(rfe.transform(RawData_Rolling_Max_Test)) # Add reduced set of features to full set 
#     print "Rolling Max With Window Size ", w, " Complete: ", int(time.time()-Start_Time), " Seconds"

    
    
    
# #Now do the same thing but for the binned time    


# Num_Bins = 200


# Train_Bin_List = []
# for i in range (0,len(Train_Array_Lengths)):
#     Train_Bin_List.append(Bin_Time(Train_Array_Lengths[i],Num_Bins)) # bin the time based on the sizes of each series. 
# Train_Time_Bins = np.concatenate(Train_Bin_List, axis = 0)


# Test_Bin_List = []
# for i in range (0,len(Test_Array_Lengths)):
#     Test_Bin_List.append(Bin_Time(Test_Array_Lengths[i],Num_Bins)) # bin the time based on the sizes of each series. 
# Test_Time_Bins = np.concatenate(Test_Bin_List, axis = 0)

# print "Time Bin Creation Complete: ", int(time.time()-Start_Time), " Seconds"

# C_Value = 1
# penalty = 'l1'
# Convergence_tol = .1

# LogReg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol) 
# rfe = RFE(LogReg,20, step=20)


# #reduce Time_Bins to top x features for HandStart
# rfe.fit(Train_Time_Bins, Train_Labels_HandStart.to_dense())
# HandStart_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
# HandStart_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
# #reduce Rolling_Means to top x features for FirstDigitTouch
# rfe.fit(Train_Time_Bins, Train_Labels_FirstDigitTouch.to_dense())
# FirstDigitTouch_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
# FirstDigitTouch_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
# #reduce Rolling_Means to top x features for BothStartLoadPhase
# rfe.fit(Train_Time_Bins, Train_Labels_BothStartLoadPhase.to_dense())
# BothStartLoadPhase_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
# BothStartLoadPhase_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
# #reduce Rolling_Means to top x features for LiftOff
# rfe.fit(Train_Time_Bins, Train_Labels_LiftOff.to_dense())
# LiftOff_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
# LiftOff_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
# #reduce Rolling_Means to top x features for Replace
# rfe.fit(Train_Time_Bins, Train_Labels_Replace.to_dense())
# Replace_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
# Replace_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set
    
# #reduce Rolling_Means to top x features for BothReleased
# rfe.fit(Train_Time_Bins, Train_Labels_BothReleased.to_dense())
# BothReleased_Train_Features.append(rfe.transform(Train_Time_Bins)) # Add reduced set of features to full set
# BothReleased_Test_Features.append(rfe.transform(Test_Time_Bins)) # Add reduced set of features to full set

Rolling Mean With Window Size  10  Complete:  52  Seconds
Rolling Variance With Window Size  10  Complete:  74  Seconds
Rolling Skew With Window Size  10  Complete:  82  Seconds
Rolling Kurtosis With Window Size  10  Complete:  89  Seconds
Rolling Min With Window Size  10  Complete:  108  Seconds
Rolling Max With Window Size  10  Complete:  142  Seconds
Rolling Mean With Window Size  50  Complete:  201  Seconds
Rolling Variance With Window Size  50  Complete:  225  Seconds
Rolling Skew With Window Size  50  Complete:  235  Seconds
Rolling Kurtosis With Window Size  50  Complete:  244  Seconds
Rolling Min With Window Size  50  Complete:  261  Seconds
Rolling Max With Window Size  50  Complete:  295  Seconds
Rolling Mean With Window Size  100  Complete:  350  Seconds
Rolling Variance With Window Size  100  Complete:  384  Seconds
Rolling Skew With Window Size  100  Complete:  397  Seconds
Rolling Kurtosis With Window Size  100  Complete:  406  Seconds
Rolling Min With Window Size  100  C

0

In [7]:
HandStart_Test_Features = np.float32(np.concatenate((HandStart_Test_Features),axis = 1))
gc.collect()

FirstDigitTouch_Test_Features = np.float32(np.concatenate((FirstDigitTouch_Test_Features),axis = 1))
gc.collect()

BothStartLoadPhase_Test_Features = np.float32(np.concatenate((BothStartLoadPhase_Test_Features),axis = 1))
gc.collect()

LiftOff_Test_Features = np.float32(np.concatenate((LiftOff_Test_Features),axis = 1))
gc.collect()

Replace_Test_Features = np.float32(np.concatenate((Replace_Test_Features),axis = 1))
gc.collect()

BothReleased_Test_Features = np.float32(np.concatenate((BothReleased_Test_Features),axis = 1))
gc.collect()

HandStart_Train_Features =np.float32( np.concatenate((HandStart_Train_Features),axis = 1))
gc.collect()

FirstDigitTouch_Train_Features = np.float32(np.concatenate((FirstDigitTouch_Train_Features),axis = 1))
gc.collect()

BothStartLoadPhase_Train_Features = np.float32(np.concatenate((BothStartLoadPhase_Train_Features),axis = 1))
gc.collect()

LiftOff_Train_Features = np.float32(np.concatenate((LiftOff_Train_Features),axis = 1))
gc.collect()

Replace_Train_Features = np.float32(np.concatenate((Replace_Train_Features),axis = 1))
gc.collect()

BothReleased_Train_Features = np.float32(np.concatenate((BothReleased_Train_Features),axis = 1))
gc.collect()




HandStart_Test_Features = np.float32(np.concatenate((HandStart_Test_Features,Test_Data),axis = 1))
gc.collect()

FirstDigitTouch_Test_Features = np.float32(np.concatenate((FirstDigitTouch_Test_Features,Test_Data),axis = 1))
gc.collect()

BothStartLoadPhase_Test_Features = np.float32(np.concatenate((BothStartLoadPhase_Test_Features,Test_Data),axis = 1))
gc.collect()

LiftOff_Test_Features = np.float32(np.concatenate((LiftOff_Test_Features,Test_Data),axis = 1))
gc.collect()

Replace_Test_Features = np.float32(np.concatenate((Replace_Test_Features,Test_Data),axis = 1))
gc.collect()

BothReleased_Test_Features = np.float32(np.concatenate((BothReleased_Test_Features,Test_Data),axis = 1))
gc.collect()
del Test_Data

HandStart_Train_Features =np.float32( np.concatenate((HandStart_Train_Features,Train_Data),axis = 1))
FirstDigitTouch_Train_Features = np.float32(np.concatenate((FirstDigitTouch_Train_Features,Train_Data),axis = 1))
BothStartLoadPhase_Train_Features = np.float32(np.concatenate((BothStartLoadPhase_Train_Features,Train_Data),axis = 1))
LiftOff_Train_Features = np.float32(np.concatenate((LiftOff_Train_Features,Train_Data),axis = 1))
Replace_Train_Features = np.float32(np.concatenate((Replace_Train_Features,Train_Data),axis = 1))
BothReleased_Train_Features = np.float32(np.concatenate((BothReleased_Train_Features,Train_Data),axis = 1))


del Train_Data


gc.collect()



0

In [9]:
print HandStart_Train_Features.shape
print HandStart_Test_Features.shape

print FirstDigitTouch_Train_Features.shape
print FirstDigitTouch_Test_Features.shape

print BothStartLoadPhase_Train_Features.shape
print BothStartLoadPhase_Test_Features.shape

print LiftOff_Train_Features.shape
print LiftOff_Test_Features.shape

print Replace_Train_Features.shape
print Replace_Test_Features.shape

print BothReleased_Train_Features.shape
print BothReleased_Test_Features.shape

NameError: name 'HandStart_Train_Features' is not defined

In [23]:
x = 934.12651

np.float32(x)

934.12653

### Maintain x% of features

based on the number of features remaining from the previous section, we can further reduce the data. 

In [8]:
#since the recursive feature elimination uses the coefficients as a means to eliminate features, 
# we must first scale the data. 
Scale_Center = StandardScaler()
Z_HandStart_Train_Features = np.float16(Scale_Center.fit_transform(np.array(HandStart_Train_Features)))
del HandStart_Train_Features 
gc.collect()


Z_HandStart_Test_Features = np.float16(Scale_Center.fit_transform(np.array(HandStart_Test_Features)))
del HandStart_Test_Features 
gc.collect()


Z_FirstDigitTouch_Train_Features = np.float16(Scale_Center.fit_transform(np.array(FirstDigitTouch_Train_Features)))
del FirstDigitTouch_Train_Features 
gc.collect()

Z_FirstDigitTouch_Test_Features = np.float16(Scale_Center.fit_transform(np.array(FirstDigitTouch_Test_Features)))
del FirstDigitTouch_Test_Features 
gc.collect()

Z_BothStartLoadPhase_Train_Features = np.float16(Scale_Center.fit_transform(np.array(BothStartLoadPhase_Train_Features)))
del BothStartLoadPhase_Train_Features 
gc.collect()

Z_BothStartLoadPhase_Test_Features = np.float16(Scale_Center.fit_transform(np.array(BothStartLoadPhase_Test_Features)))
del BothStartLoadPhase_Test_Features 
gc.collect()



Z_LiftOff_Train_Features = np.float16(Scale_Center.fit_transform(np.array(LiftOff_Train_Features)))
del LiftOff_Train_Features 
gc.collect()

Z_LiftOff_Test_Features = np.float16(Scale_Center.fit_transform(np.array(LiftOff_Test_Features)))
del LiftOff_Test_Features 
gc.collect()



Z_Replace_Train_Features = np.float16(Scale_Center.fit_transform(np.array(Replace_Train_Features)))
del Replace_Train_Features 
gc.collect()

Z_Replace_Test_Features = np.float16(Scale_Center.fit_transform(np.array(Replace_Test_Features)))
del Replace_Test_Features 
gc.collect()



Z_BothReleased_Train_Features = np.float16(Scale_Center.fit_transform(np.array(BothReleased_Train_Features)))
del BothReleased_Train_Features 
gc.collect()

Z_BothReleased_Test_Features = np.float16(Scale_Center.fit_transform(np.array(BothReleased_Test_Features)))
del BothReleased_Test_Features 
gc.collect()


0

In [18]:
print HandStart_PCs_Train.shape

(391450L, 58L)


In [17]:
Pct_Variance_Explained = .9
Start_Time = time.time()

HandStart_PCs_Train,HandStart_PCs_Test = extractFeatures_PCA(Z_HandStart_Train_Features,Z_HandStart_Test_Features,Pct_Variance_Explained)
del Z_HandStart_Train_Features,Z_HandStart_Test_Features 
gc.collect()
print "HandStart_PCs Complete:", int(time.time()-Start_Time), " Seconds"

FirstDigitTouch_PCs_Train, FirstDigitTouch_PCs_Test = extractFeatures_PCA(Z_FirstDigitTouch_Train_Features,Z_FirstDigitTouch_Test_Features,Pct_Variance_Explained)
del Z_FirstDigitTouch_Train_Features,Z_FirstDigitTouch_Test_Features
gc.collect()
print "FirstDigitTouch_PCs Complete:", int(time.time()-Start_Time), " Seconds"

BothStartLoadPhase_PCs_Train, BothStartLoadPhase_PCs_Test =extractFeatures_PCA(Z_BothStartLoadPhase_Train_Features,Z_BothStartLoadPhase_Test_Features,Pct_Variance_Explained)
del Z_BothStartLoadPhase_Train_Features,Z_BothStartLoadPhase_Test_Features
gc.collect()
print "BothStartLoadPhase_PCs Complete:", int(time.time()-Start_Time), " Seconds"

LiftOff_PCs_Train, LiftOff_PCs_Test = extractFeatures_PCA(Z_LiftOff_Train_Features,Z_LiftOff_Test_Features,Pct_Variance_Explained)
del Z_LiftOff_Train_Features,Z_LiftOff_Test_Features
gc.collect()
print "LiftOff_PCs Complete:", int(time.time()-Start_Time), " Seconds"

Replace_PCs_Train, Replace_PCs_Test = extractFeatures_PCA(Z_Replace_Train_Features,Z_Replace_Test_Features,Pct_Variance_Explained)
del Z_Replace_Train_Features,Z_Replace_Test_Features
gc.collect()
print "Replace_PCs Complete:", int(time.time()-Start_Time), " Seconds"


BothReleased_PCs_Train, BothReleased_PCs_Test = extractFeatures_PCA(Z_BothReleased_Train_Features,Z_BothReleased_Test_Features,Pct_Variance_Explained)
del Z_BothReleased_Train_Features,Z_BothReleased_Test_Features
gc.collect()
print "BothReleased_PCs Complete:", int(time.time()-Start_Time), " Seconds"





HandStart_PCs Complete: 15  Seconds
FirstDigitTouch_PCs Complete: 30  Seconds
BothStartLoadPhase_PCs Complete: 44  Seconds
LiftOff_PCs Complete: 60  Seconds
Replace_PCs Complete: 75  Seconds
BothReleased_PCs Complete: 89  Seconds


In [ ]:
Percent_Features_To_Keep = .7
Num_Steps = 2
Num_Features = Z_BothReleased_Test_Features.shape[1]

C_Value = 1
penalty = 'l1'
Convergence_tol = .01

Num_Features_To_Keep = int(Num_Features*Percent_Features_To_Keep)
RFE_Step = int((Num_Features-Num_Features_To_Keep)/Num_Steps)

Start_Time = time.time()

LogReg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol) 
rfe = RFE(LogReg,Num_Features_To_Keep, step=RFE_Step)

#reduce entire feature set to top x features for HandStart
rfe.fit(Z_HandStart_Train_Features, Train_Labels_HandStart.to_dense())
HandStart_Train_Reduced_Features = np.float16(rfe.transform(Z_HandStart_Train_Features))
del Z_HandStart_Train_Features
gc.collect()
print "HandStart_Train_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"

HandStart_Test_Reduced_Features = np.float16(rfe.transform(Z_HandStart_Test_Features))
del Z_HandStart_Test_Features
gc.collect()
print "HandStart_Test_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"
    
#reduce entire feature set  to top x features for FirstDigitTouch
rfe.fit(Z_FirstDigitTouch_Train_Features, Train_Labels_FirstDigitTouch.to_dense())
FirstDigitTouch_Train_Reduced_Features = np.float16(rfe.transform(Z_FirstDigitTouch_Train_Features))
del Z_FirstDigitTouch_Train_Features
gc.collect()
print "FirstDigitTouch_Train_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"

FirstDigitTouch_TestReduced__Features = np.float16(rfe.transform(Z_FirstDigitTouch_Test_Features))
del Z_FirstDigitTouch_Test_Features
gc.collect()
print "FirstDigitTouch_TestReduced__Features Complete:", int(time.time()-Start_Time), " Seconds"

    
#reduce entire feature set  to top x features for BothStartLoadPhase
rfe.fit(Z_BothStartLoadPhase_Train_Features, Train_Labels_BothStartLoadPhase.to_dense())
BothStartLoadPhase_Train_Reduced_Features = np.float16(rfe.transform(Z_BothStartLoadPhase_Train_Features))
del Z_BothStartLoadPhase_Train_Features
gc.collect()
print "BothStartLoadPhase_Train_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"

BothStartLoadPhase_Test_Reduced_Features = np.float16(rfe.transform(Z_BothStartLoadPhase_Test_Features)) 
del Z_BothStartLoadPhase_Test_Features
gc.collect()
print "BothStartLoadPhase_Test_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"


#reduce entire feature set  to top x features for LiftOff
rfe.fit(Z_LiftOff_Train_Features, Train_Labels_LiftOff.to_dense())
LiftOff_Train_Reduced_Features = np.float16(rfe.transform(Z_LiftOff_Train_Features))
del Z_LiftOff_Train_Features
gc.collect()
print "LiftOff_Train_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"

LiftOff_Test_Reduced_Features = np.float16(rfe.transform(Z_LiftOff_Test_Features)) 
del Z_LiftOff_Test_Features
gc.collect()
print "LiftOff_Test_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"

    
#reduce entire feature set  to top x features for Replace
rfe.fit(Z_Replace_Train_Features, Train_Labels_Replace.to_dense())
Replace_Train_Reduced_Features = np.float16(rfe.transform(Z_Replace_Train_Features) )
del Z_Replace_Train_Features
gc.collect()
print "Replace_Train_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"

Replace_Test_Reduced_Features = np.float16(rfe.transform(Z_Replace_Test_Features))
del Z_Replace_Test_Features
gc.collect()
print "Replace_Test_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"

    
#reduce entire feature set  to top x features for BothReleased
rfe.fit(Z_BothReleased_Train_Features, Train_Labels_BothReleased.to_dense())
BothReleased_Train_Reduced_Features = np.float16(rfe.transform(Z_BothReleased_Train_Features))
del Z_BothReleased_Train_Features
gc.collect()
print "BothReleased_Train_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"

BothReleased_Test_Reduced_Features = np.float16(rfe.transform(Z_BothReleased_Test_Features))
del Z_BothReleased_Test_Features
gc.collect()
print "BothReleased_Test_Reduced_Features Complete:", int(time.time()-Start_Time), " Seconds"
   





# Test Different Models

##Logistic Regession

In [20]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#set parameters for Logistic Regression
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
penalty = 'l1'
Convergence_tol = .01

#Create a function that trains and runs a logistic regression model. Then prints and returns the AUC score
def predictCategory_LogReg(train_data, train_label, test_data, test_label, Category, C_Value,penalty, Convergence_tol):
    Logistic_Reg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol)    
    Logistic_Reg.fit(train_data, train_label)
    print(Category, " Score =", Logistic_Reg.score(test_data, test_label))

    prob = Logistic_Reg.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])

Start_Time = time.time()   
AUC_HandStart = predictCategory_LogReg(HandStart_PCs_Train,Train_Labels_HandStart.to_dense(),HandStart_PCs_Test,Test_Labels_HandStart.to_dense(), 'HandStart', C_Value,penalty, Convergence_tol)
AUC_FirstDigitTouch = predictCategory_LogReg(FirstDigitTouch_PCs_Train,Train_Labels_FirstDigitTouch.to_dense(),FirstDigitTouch_PCs_Test,Test_Labels_FirstDigitTouch.to_dense(), 'FirstDigitTouch', C_Value,penalty, Convergence_tol)
AUC_BothStartLoadPhase = predictCategory_LogReg(BothStartLoadPhase_PCs_Train,Train_Labels_BothStartLoadPhase.to_dense(),BothStartLoadPhase_PCs_Test,Test_Labels_BothStartLoadPhase.to_dense(), 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)
AUC_LiftOff = predictCategory_LogReg(LiftOff_PCs_Train,Train_Labels_LiftOff.to_dense(),LiftOff_PCs_Test,Test_Labels_LiftOff.to_dense(), 'LiftOff', C_Value,penalty, Convergence_tol)
AUC_Replace = predictCategory_LogReg(Replace_PCs_Train,Train_Labels_Replace.to_dense(),Replace_PCs_Test,Test_Labels_Replace.to_dense(), 'Replace', C_Value,penalty, Convergence_tol)
AUC_BothReleased = predictCategory_LogReg(BothReleased_PCs_Train,Train_Labels_BothReleased.to_dense(),BothReleased_PCs_Test,Test_Labels_BothReleased.to_dense(), 'BothReleased', C_Value,penalty, Convergence_tol)
print int(time.time()-Start_Time), " Seconds to complete"
print "Overall Logistic Regression Score = ", np.mean((AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased))

('HandStart', ' Score =', 0.98024024189620151)
('HandStart', 'AUC', 0.72901344112293254)
('FirstDigitTouch', ' Score =', 0.98007021607065725)
('FirstDigitTouch', 'AUC', 0.71988603193346079)
('BothStartLoadPhase', ' Score =', 0.98023105131103694)
('BothStartLoadPhase', 'AUC', 0.72491754512914097)
('LiftOff', ' Score =', 0.98045622064756865)
('LiftOff', 'AUC', 0.73913876482865626)
('Replace', ' Score =', 0.97736818403227732)
('Replace', 'AUC', 0.67686674000946967)
('BothReleased', ' Score =', 0.97745549459134062)
('BothReleased', 'AUC', 0.6705658987427523)
109  Seconds to complete
Overall Logistic Regression Score =  0.710064736961


### This is a failed attempt at parallel processing the model 

In [ ]:
import multiprocessing as mp
from multiprocessing import Process
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#set parameters for Logistic Regression
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
penalty = 'l1'
Convergence_tol = .01

#Create a function that trains and runs a logistic regression model. Then prints and returns the AUC score
def predictCategory_LogReg(train_data, train_label, test_data, test_label, Category, C_Value,penalty, Convergence_tol):
    Logistic_Reg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol)    
    Logistic_Reg.fit(train_data, train_label)
    print(Category, " Score =", Logistic_Reg.score(test_data, test_label))

    prob = Logistic_Reg.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])

Start_Time = time.time()
def Log_Reg_HandStart():
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', C_Value,penalty, Convergence_tol)

    
def Log_Reg_FirstDigitTouch():    
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', C_Value,penalty, Convergence_tol)

    
def Log_Reg_BothStartLoadPhase():       
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)

    
def Log_Reg_LiftOff():       
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', C_Value,penalty, Convergence_tol)

    
def Log_Reg_Replace():     
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', C_Value,penalty, Convergence_tol)

    
def Log_Reg_BothReleased():    
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', C_Value,penalty, Convergence_tol)

    

def Overall_AUC(i):
    if i == 1:
        Log_Reg_HandStart()
    if i == 2:
        Log_Reg_FirstDigitTouch()
    if i == 3:
        Log_Reg_BothStartLoadPhase()
    if i == 4:
        Log_Reg_LiftOff()
    if i == 5:
        Log_Reg_Replace()
    if i == 6:
        Log_Reg_BothReleased()
    
# print Overall_AUC(AUC_Values)


   
    
pool = mp.Pool(processes=1)
results = [pool.apply_async(Overall_AUC, args=(i,)) for i in range(1,7)]
output = [p.get() for p in results]
print(output)



print time.time()-Start_Time






## SVM Model

This is set up to get started. It has not been tested. MM 08/04/2015 11:30 A.M.

In [ ]:
#set parameters for SVM
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
kernel = 'rbf'
degree = 3

#Create a function that trains and runs a model. Then prints and returns the AUC score
def predictCategory_SVM(train_data, train_label, test_data, test_label, Category, C_Value,kernel, degree):
    SVM = SVC(C = C_Value, kernel = kernel,degree=degree)    
    SVC.fit(train_data, train_label)

    prob = SVC.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])
    
AUC_HandStart = predictCategory_SVM(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', Category, C_Value,kernel, degree)
AUC_FirstDigitTouch = predictCategory_SVM(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', Category, C_Value,kernel, degree)
AUC_BothStartLoadPhase = predictCategory_SVM(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', Category, C_Value,kernel, degree)
AUC_LiftOff = predictCategory_SVM(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', Category, C_Value,kernel, degree)
AUC_Replace = predictCategory_SVM(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', Category, C_Value,kernel, degree)
AUC_BothReleased = predictCategory_SVM(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', Category, C_Value,kernel, degree


print "Overall SVM Score = ", np.mean(AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased)

##Neural Network

In [ ]:
#set parameters for Neural Network model
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
n_components = 10
learning_rate = .1
batch_size = 20000
n_iter =1


#Create a function that trains and runs a model. Then prints and returns the AUC score
def predictCategory_Neural_Network(train_data, train_label, test_data, test_label, Category, C_Value,kernel, degree):
    SVM = SVC(C = C_Value, kernel = kernel,degree=degree)    
    SVC.fit(train_data, train_label)

    prob = SVC.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])
    
AUC_HandStart = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', Category, C_Value,kernel, degree)
AUC_FirstDigitTouch = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', Category, C_Value,kernel, degree)
AUC_BothStartLoadPhase = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', Category, C_Value,kernel, degree)
AUC_LiftOff = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', Category, C_Value,kernel, degree)
AUC_Replace = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', Category, C_Value,kernel, degree)
AUC_BothReleased = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', Category, C_Value,kernel, degree


print "Overall Neural Network Score = ", np.mean(AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased)


## Decision Trees and Random Forest